# Topic analysis based on Genius songs lyrics

Author : lievre.thomas@gmail.com

---

In this notebook, we will explore the genius data extract from [Kaggle](https://www.kaggle.com/datasets/carlosgdcj/genius-song-lyrics-with-language-information).

**The aim of this analysis is to retrieve topic from lyrics and retrieve main topics by year or decade.**

This notebook was carried out in the context of a class project imposed by the [text mining course (TDDE16)](https://www.ida.liu.se/~TDDE16/project.en.shtml) of Linköpings universitet.


## Few informations about Genius website

Genius is an American digital company founded on August 27, 2009, by Tom Lehman, lan Zechory, and Mahbod Moghadam. Originally launched as Rap Genius with a focus on hip-hop music, it was initially a crowdsourced website where people could fill in the lyrics of rap music and give an interpretation of the lyrics. Over the years the site has grown to contain several million annotated texts from all eras ( from [Wikipedia Genius page](https://en.wikipedia.org/wiki/Genius_(company))).


## Load the data in memory

Data are all contain in a big 9GB csv file (around 5 millions rows). It could be dificult to load all this data in our computer memory. To deal with this issue, I made a loading class to split the data in 6 pickles files to improve the compressness of the data which aim to improve the loading speed in the memory. Then the pickles are randomly draw to improve generality of the data. We currently assumed the data pickles batch are identically distributed (we will explore the data batches at the second part). The class below deal with all the process.

In [ ]:
import pandas as pd
from random import seed, sample
import pickle
import glob
import os

class Loader():

    def __init__(self, in_path, out_path):
        """
        Args:
            in_path (str): csv input path.
            out_path (str): Output directory path to store the pickles.
            chunksize (int, optional): Chunksize for DataFrame reader. Defaults to 10**6. 
        """

        self.__in_path = in_path
        self.__out_path = out_path
        self.__chunksize = 10**6

    def __produce_pickles(self):
        """produce pickles by reading csv by chunksize
        """
        with pd.read_csv(self.__in_path, chunksize = self.__chunksize) as reader:
            try:
                os.makedirs(self.__out_path)
            except FileExistsError:
                # directory already exists
                pass
            for i, chunk in enumerate(reader):
                out_file = self.__out_path + "/data_{}.pkl".format(i+1)
                with open(out_file, "wb") as f:
                    pickle.dump(chunk, f, pickle.HIGHEST_PROTOCOL)
    
    def load_pickle(self, pickle_id):
        """load a pickle file by id

        Args:
            pickle_id (int): pickle id.

        Raises:
            Exception: The path of the given id isn't a file

        Returns:
            obj: DataFrame
        """
        # produce the pickles if the directory not exists or
        # if the directory is empty 
        if (not os.path.exists(self.__out_path)) or \
              (len(os.listdir(self.__out_path)) == 0):
            self.__produce_pickles()
        
        # get the file path following the pickle_id
        # given in parameter
        file_path = self.__out_path + \
            "/data_" + str(pickle_id) + ".pkl"

        if os.path.isfile(file_path):
            df = pd.read_pickle(file_path)
        else:
            raise Exception("The pickle file data_{}.pkl doesn't exist".format(pickle_id))
        return df
        

    def random_pickles(self, n_pickles = 3, init = 42, verbose = True):
        """random reader over pickles files

        Args:
            n_pickles (int, optional): number of pickles to load. Defaults to 3.
            init (int, optional): Integer given to the random seed. Defaults to 42.
            verbose (bool, optional): Print the loaded files. Defaults to True

        Raises:
            Exception: Stop the process if n_pickles exceed pickle files number.

        Returns:
            obj: pd.Dataframe
        """

        # produce the pickles if the directory not exists or
        # if the directory is empty 
        if (not os.path.exists(self.__out_path)) or \
              (len(os.listdir(self.__out_path)) == 0):
            self.__produce_pickles()

        pickle_files = [name for name in
                        glob.glob(self.__out_path + "/data_*.pkl")]
        # draw p_files        
        seed(init)

        if n_pickles <= 6:
            random_p_files = sample(pickle_files, n_pickles)
        else:
            raise Exception("The parameter n_pickles (" +
                            "{}) exceed the numbers of pickle files ({})"\
                                .format(n_pickles, len(pickle_files)))
        # print the drawed files
        if verbose:
            print("Loaded pickles:")
            for p in random_p_files:
                print(p)

        # load random pickles file
        df_list = [pd.read_pickle(p) for p in random_p_files]

        # create the dataframe by concatenate the previous
        # dataframes list
        df = pd.concat(df_list, ignore_index = True)
        return df

In [ ]:
# create reader
#  /!\ change path in kaggle
kaggle_input = "/kaggle/input/genius-song-lyrics-with-language-information/song_lyrics.csv"
kaggle_output = "/kaggle/working/data/"

# initiate the file loader
loader = Loader(in_path = kaggle_input, out_path = kaggle_output)

# load random pickle files
df = loader.random_pickles(n_pickles = 1)

Batchs of data are randomly loaded in the memory. The number of batchs loaded depends on the memory capacity of the computer running the script. For the analysis, we will only works on the random samples loaded (All the data in Kaggle).  

## Exploring the coarse data

Let's visualize and explore the coarse data before a part of deeper analysis.

In [ ]:
df.head()

For each songs, we've got several informations :
- title of the song
- the tag (which kind of music)
- the artist singer name
- the release year
- the number of page views
- the featuring artists names
- the lyrics
- the genius identifier
- Lyrics language according to [CLD3](https://github.com/google/cld3). Not reliable results are NaN. CLD3 is a neural network model for language indentification.
- Lyrics language according to [FastText's langid](https://fasttext.cc/docs/en/language-identification.html). Values with low confidence (<0.5) are NaN. FastText's langid is library developped by Facebook’s AI Research lab for efficient learning of word representations and sentence classification. fastText has also published a fast and accurate tool for text-based language identification capable of recognizing more than 170 languages.
- Combines language_cld3 and language_ft. Only has a non NaN entry if they both "agree".

More information at this link : https://www.kaggle.com/datasets/carlosgdcj/genius-song-lyrics-with-language-information

In [ ]:
df.dtypes

In [ ]:
# display the size
print('Data frame size (row x columns):', df.size)
print('Data rows number: ', len(df))
print('Number of unique songs (following genius id): ', len(df.id.unique()))

Genius id seems to be the unique rows identifier.

Let's vizualise size of the coarse data over years before preprocessing to compare batch distributions. One things to know before vizualise the data, the pickles are create by chunks reading. 

The last diplayed table gives us some information about the data. The csv file seems to be sort by id, so the pickle files are then sort too.

In [ ]:
import plotly.express as px
import plotly.io as pio

# change theme template for every graph below
pio.templates.default = "plotly_white"


# get some information about the pickle data
def pickle_informations(data = loader):
    rows = []
    for i in range(1, len(os.listdir('data')) + 1):
        df = data.load_pickle(i)
        rows.append(len(df))
        del df
    return rows

# get the rows
rows = pickle_informations()

# create the dataframe
df_data = pd.DataFrame(
    {'batch' : ['data ' + str(i) for i in range(1,len(rows) + 1)],
    'rows' : rows})

fig = px.bar(df_data, x="batch", y="rows")
fig.show()

Batch seems to have the same number of rows rexcept for the last one which is consistent because batch are create iteratively by 10e6 chunks over the csv The last batch could be seen as a rest.

In [ ]:
import plotly.graph_objects as go

def add_bar(i, y1, y2, color, data = loader):
    df = data.load_pickle(i)
    df = df[(df.year >= y1) & (df.year <= y2)]
    df_year = df.groupby(['year']).size().reset_index(name='count')
    new_bar = go.Bar(
                x = df_year.year.values,
                y = df_year['count'].values,
                name = 'data_'+ str(i),
                marker = {'color' : color})
    new_trend = go.Scatter(
                x = df_year.year.values,
                y = df_year['count'].values,
                mode="lines",
                line={'color' : color,
                    'width' : 0.5},
                showlegend=False)
    del df_year, df
    return new_bar, new_trend


def multi_barplot(year1, year2, colors):    
    # create a empty plotly.Figure object
    fig = go.Figure() 
    # compute the batch number
    n_batch = len(os.listdir('data'))
    # test the color list feed in argument
    # fit well with the batch number
    if n_batch > len(colors):
        raise Exception(
            "The colors list size({})doesn't ".format(len(colors)) +
            "fit with the number of data".format(n_batch))
    for i in range(1, n_batch + 1):
        fig.add_traces((add_bar(i, year1, year2, colors[i-1])))
    fig.update_layout(
        title = "Data distribution over years ({} - {})"
            .format(year1, year2),
        xaxis_title="years",
        yaxis_title="title",
        legend_title="Data batch")
    return fig


In [ ]:
import plotly.colors as col

# create the color list
colors = col.qualitative.Plotly

# 1990 - 2023
fig1 = multi_barplot(1960, 1989, colors)
fig1.show()
# 1960 - 1990
fig2 = multi_barplot(1990, 2023, colors)
fig2.show()

The first bar chart (1960 - 1989) shows an increasing numbers of data over years. Moreover batch seems to have quite similar distriutions over years. data_1 and data_2 batch quite outperform the 4 others. data_6 batch is weaker than the other due to its poor number of rows.
The data behaves similarly until 2012 as we can see on the second chart (1990-2023). After this year there is great increasing of the data retrieved. A minimum increase of at least 100% of the batch can be observed. An increase of up to 50 times the batch size for some.

## Data pre-processing

The aim of this part is to preprocess data in order to get suitable data for the analysis. let's focus on the year variable.

We will focus on English songs, to facilitate the analysis and the work of natural language processing algorithms.

In [ ]:
# Retrieve only the texts identified as English language by both cld3 and fasttext langid
df = df[df.language == 'en']

Next, it can be quite interseting to check Nan values

In [ ]:
# find which column contain nan value
df.columns[df.isna().any()].tolist()

In [ ]:
# get all rows that contain NaN values
df_nan = df[df.isna().any(axis=1)]
df_nan

In [ ]:
print('Number of untitled song:', len(df[df.isna().any(axis=1)]))

Insofar as the title of the music is not to be taken into account in the learning of the topic modeling algorithms but But the titles can be related to the topics in the next phase of analysis and the low number of songs without any title, I decide to delete this data for the moment.

In [ ]:
# Delete rows containing NaN values
df = df.dropna()
len(df)

Next, we also try to check for None values

In [ ]:
df[df.isnull().any(axis=1)]

No None values in this dataframe.

Afterwards, let's look at the year variable, which is one of the important variables to take into account in our analysis because we want to extract the topics by decades.

In [ ]:
years = df.year.unique()
print(years)

print('Number of unique years: ',len(years))

We firstly want to know if the year variable format is suitable. It is highly likely that year are sometimes downsized (example : 92 instead of 1992).
Let's display the tag distribution for music with a release year below 215.

In [ ]:
df_tag = df[df['year'] < 215].groupby(['tag']).size().reset_index(name='count')

fig = px.pie(df_tag, names="tag", values="count", title = "Outlier tag distribution")
fig.show()

It is rather surprising to observe that the majority style of music of this period (< 215) is rap music knowing that this style is known for the current emerging style. Of course, among this data their is a important part of outlier year.

In [ ]:
# Extract the pieces of music of type 'rap' lower than the year 215
df_rap = df[(df['year'] < 215) & (df['tag'] == 'rap')]
df_rap.sort_values(by='views',ascending=False).head(20)

In [ ]:
df_rap[df_rap['artist'] == 'Kanye East']

If we search the release date of this track on google, we can find a release date from 4 May 2021 on the [Genius website](https://genius.com/Kanye-east-the-secrets-of-dababy-lyrics). Given the year that we find in our table and real one, we can assume some issue about the date format (1 instead 2021).

After few research on genius website, the most viewed songs of this above displayed list seems to be released on 2021 but more views decrease harder is the interpretation of date.

Let's check the second most popular tag 'pop' in this retrieve outliers data :

In [ ]:
# Extract the pieces of music of type 'rap' lower than the year 215
df_pop = df[(df['year'] < 215) & (df['tag'] == 'pop')]
df_pop.sort_values(by='views',ascending=False).head(20)

The titles recovered seem to be for the most part recent sounds, not very popular with a bad indexation of the years.

A case-by-case pre-processing of the data is too tedious compared to the amount of data to be processed. We will only use data with correctly formatted dates.

In [ ]:
df = df[(df.year >= 1960) & (df.year < 2023)]
len(df)

We wish to analyze the texts by decade then let's add a decade column.

In [ ]:
import math

df['decade'] = df['year'].map(lambda x : int(math.trunc(x / 10) * 10))

df.sort_values(by = 'year').head(20)

## Data vizualisation

Let's do some vizualisation to get a better understanding of our data. As we saw on previous distribution graphs over years, more titles have been recorded over the last 2 decades.

In [ ]:
# barplot by decade
def barplot_by_decade(df):

    # groupby decade
    df_d = df.groupby(['decade']).size().reset_index(name='count')

    # create the figure
    fig = go.Figure()

    fig.add_bar(
        x=df_d.decade,
        y=df_d['count'],
        showlegend=False)

    fig.add_scatter(
            x=df_d.decade,
            y=df_d["count"],
            mode="markers+lines",
            name="trend",
            showlegend=False)

    fig.update_layout(
            title = "Music release over years",
            xaxis_title="decade",
            yaxis_title="release")
    return fig

# build and display
fig = barplot_by_decade(df)
fig.show()

In [ ]:
# compute tag frequencies by decade
df_pies_d = df.groupby(['decade','tag']).size().reset_index(name='count')
df_pies_d[df_pies_d.decade == 1960]

In [ ]:
from plotly.subplots import make_subplots

# create en make subplot
fig = make_subplots(rows=3, cols=3,
                    specs=[
                        [{'type':'domain'}
                        for i in range(1,4)] for i in range(1,4)
                    ])
decades = df_pies_d.decade.unique().tolist()
for i in range(0,3):
    for k in range(0,3):
        decade = decades[i*3 + k]
        # group by decade
        df_p = df_pies_d[df_pies_d.decade == decade]
        # add figure
        fig.add_trace(go.Pie(labels=df_p.tag, values=df_p['count'], name=decade), i+1, k+1)
        # add annotation
        fig.add_annotation(arg=dict(
            text=decade, x=k*0.375 + 0.125,
            y=-i*0.3927 + 0.90, font_size=10,
            showarrow=False))
        if (i*3 + k) == 6:
            break


# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.4, hoverinfo="label+percent+name")

fig.update_layout(
    title_text="Tags proportions by decades"
    # Add annotations in the center of the donut pies.
    #annotations=[dict(text=decade, x=k*0.375+0.125, y= -i*0.125+0.90, font_size=10, showarrow=False)
     #           for k, decade in enumerate(decades) for i in range(0,4)]
)
fig.show()

The different pie charts show us an evolution of the different proportions of music styles by decades from 1960 to today. In the sixties the most listed music style is the **pop music**. Then comes an emergence of a rising style: the **rock**. This one increases with the decades until it overtakes the pop music during the decades 90 and 2000. The two styles come to balance thereafter facing the meteoric rise of the most listed genre in our current decade, namely **rap**. The evolution of the proportions recover can give us a rather precise idea of the most popular styles of their times. However, it is important to remember that the data still has some bias, Genius being a crowdsourced tool created during the last decade and at the base only as a lexical translator of rap music, it is normal to find a large amount of data from this period and especially from this style there.

## Text preprocessing

After the visualisation part let's focus more on the main data which are the lyrics.

In [ ]:
df.iloc[0]["lyrics"]

There is many undesirable characters like the line breaker '\n', figures or square, curly and simple brackets. So let's clean this data with regular expressions.

In [ ]:
import re
from numpy.random import randint

def clean_text(text):
    # remove \n
    text = text.replace('\n', '')
    # remove punctuation
    text = re.sub(r'[,\.!?]', '', text)
    #removing text in square braquet
    text = re.sub(r'\[.*?\]', ' ', text)
    #removing numbers
    text = re.sub(r'\w*\d\w*',' ', text)
    #removing bracket
    text = re.sub(r'[()]', ' ', text)
    # convert all words in lower case
    text = text.lower()
    return text

# get the results of data cleaning
cleaned_text = df["lyrics"].apply(clean_text)

In [ ]:
docs = cleaned_text.to_list()
docs[0]

In [ ]:
# update dataframe
df.update(cleaned_text)
df.head(3)

That's better! The libraries that we will use later to perform topic modeling usually provide preprocessing but it is always good to have control over what we manipulate.

## Topic modeling

I will perform 2 ways to do topic modeling :
- [LDA (latent dirichlet allocation)](https://fr.wikipedia.org/wiki/Allocation_de_Dirichlet_latente) are the common way to do topic modeling in the few last years, it works and it's quite easy to use with common python library like [Gensim](https://radimrehurek.com/gensim/auto_examples/index.html).
- [BERTopic](https://maartengr.github.io/BERTopic/index.html) seems to be one of the best technic this day to perform topic modeling. It combine the leverage of [BERT](https://en.wikipedia.org/wiki/BERT_(language_model)) the famous language model with [c-TF-IDF](https://maartengr.github.io/BERTopic/api/ctfidf.html) tansformer. 


### Define default tokenizer and Lemmatizer

In [ ]:
import spacy

print("set gpu: ", spacy.prefer_gpu())

# small model /!\ take the bigger one for Kaggle
new_nlp = spacy.load('en_core_web_sm')

It could be difficult to process all this data on my computer or Kaggle. The memory will quickly be overwhelmed. I will work with a sample of our previously load data in order to avoid memory overload.

In [ ]:
# set sample frac of the data
prop = 0.1

# create sample df 1/3 of the actual loaded data
sdf = df.sample(frac = prop)

In [ ]:
# check the distribution of the sample
barplot_by_decade(sdf)

In [ ]:
# default preprocessing
def preprocess(text, nlp = new_nlp):

    #TOKENISATION
    tokens =[]
    for token in nlp(text):
        tokens.append(token)

    #REMOVING STOP WORDS
    spacy_stopwords = new_nlp.Defaults.stop_words
    sentence =  [word for word in tokens if word.text.isalpha() and word.text not in spacy_stopwords]

    #LEMMATISATION
    sentence = [word.lemma_ for word in sentence]

    return sentence

### Vizualise most frequent words over decades

In [ ]:
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer

class TermsDocumentsMatrix():
    
    def __init__(self, sdf, decades=[1960, 1970], colorscale = 'Plotly3'):
        # vectorizer on the sample lyrics
        self.__vectorizer = CountVectorizer(tokenizer = preprocess)
        # fit and transform the data
        self.__data_vectorized = self.__vectorizer.fit_transform(
            tqdm(sdf['lyrics'].loc[sdf['decade'].isin(decades)])
        )
        # get decades informations
        self.__decades = sdf['decade'].loc[sdf['decade'].isin(decades)].reset_index(drop=True)
        self.__unique_decades = decades
        # get colorscale template
        self.__colorscale = colorscale
    
    def get_tdmatrix(self):
        
        # compute a Matrix terms document by decades
        df_bw = pd.DataFrame(self.__data_vectorized.toarray(),
                    columns = self.__vectorizer.get_feature_names_out())
        
        # check the length
        if len(df_bw) != len(self.__decades):
            raise Exception('Not the same size')
        
        # concatenate decade
        df_bw['decade'] = self.__decades
        
        # check NaN values
        if len(df_bw.columns[df_bw.isna().any()].tolist()) != 0:
            raise Exception('Decade got Nan values')

        return df_bw
    
    def get_tdm_by_decade(self, decade):
        
        if decade not in self.__unique_decades:
            raise Exception("{} doesn't appear in the decades list".format(decade))
        
        # compute a Matrix terms document by decades (bag of words format)
        df_bw = pd.DataFrame(self.__data_vectorized.toarray(),
                    columns = self.__vectorizer.get_feature_names_out())
        
        # check the length
        if len(df_bw) != len(self.__decades):
            raise Exception('Not the same size')
        
        # concatenate decade
        df_bw['decade'] = self.__decades
        
        # check NaN values
        if len(df_bw.columns[df_bw.isna().any()].tolist()) != 0:
            raise Exception('Decade got Nan values')
        
        # select suitable decade
        df_bw = df_bw[df_bw['decade'] == decade]
        
        return df_bw
    
    def most_freq_terms(self, n_rows = 1, n_cols = 2, n_terms = 10):
        
        # create the document terms matrix
        df_bw = self.get_tdmatrix()
        
        # create en make subplot
        fig = make_subplots(rows=n_rows, cols=n_cols,
                            x_title = 'number of occurrences',
                            y_title = 'terms',
                            subplot_titles = self.__unique_decades)
        
        for i in range(0,n_rows):
            for k in range(0,n_cols):
                if (i*n_rows + k) == len(self.__unique_decades):
                    break
                
                # get the decade
                decade = self.__unique_decades[i*n_rows + k]
            
                #select the suitable decade and delete decade column
                df_decade = df_bw.loc[df_bw.decade == decade, df_bw.columns != 'decade']
                
                # compute terms frequencies by decade
                terms_freq = df_decade.sum().sort_values(ascending = False)
            
                # total number of terms occurences
                total_terms = terms_freq.values
                
                # add figure
                fig.add_trace(go.Bar(y=terms_freq.index.tolist()[:n_terms][::-1],
                                     x=total_terms[:n_terms][::-1],
                                     name=decade,
                                     orientation='h', showlegend = False,
                                    marker = dict(color = total_terms,
                                                  colorscale=self.__colorscale)),
                              i+1, k+1)
        return fig

In [ ]:
# first decades
tdm = TermsDocumentsMatrix(sdf, decades = [1960, 1970],
                           colorscale = 'Plotly3')

# display bar charts of most frequent terms
tdm.most_freq_terms(n_rows = 1, n_cols = 2, n_terms = 15)

According to the bar graphs displayed above, a group of words seems to recur on each decade: Love, know, go, feel ... Words that seem to relate to the popular song that can talk about love. This is consistent with our previous analysis from the pie charts showing the proportions of musical styles across time. We also notice an important presence of onomatopoeia like yeah or oh.

In [ ]:
# first decades
tdm = TermsDocumentsMatrix(sdf, decades = [2020],
                          colorscale = 'Plasma')

# most frequent terms
tdm.most_freq_terms(n_rows = 1, n_cols = 1, n_terms = 15)

We get similar results on this second decade with similar high occurrence words. We see a greater amount of onomatopoeia in the current decade. We can explain this by an emergence of the rap music style on this current and last decade. There is in this style of music a very used process, the 'ad-libs'. They are sounds, words or onomatopoeias that the artists pronounce sometimes between two verses or at the end of a sentence to give more impact to their text and to dynamize the atmosphere of a title. This may explain the greater presence of onomatopoeia in the lyrics of this decade.


### Topic modeling with LDA

LDA is a common technic use in topic modeling, we firstly process basic preprocessing.

In [149]:
# gensim
from gensim.models import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
from gensim.test.utils import datapath

# utils
from datetime import datetime

# dashboards
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt

# TSNE dependencies
from sklearn.manifold import TSNE
from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook
import numpy as np
import matplotlib.colors as mcolors


class LDATopicModeling():
    
    def __init__(self, df = sdf,
                 decade = 1960,
                 directory = "/kaggle/working/models/",
                 existing = False,
                 n_topics = 10,
                lang_preprocess = preprocess):
        # Apply preprocessing on decade data
        documents = df.loc[df.decade == decade, 'lyrics'].apply(lang_preprocess)
            
        # Create a corpus from a list of texts
        self.__id2word = Dictionary(documents.tolist())
        self.__corpus = [self.__id2word.doc2bow(doc) for doc in documents]
        self.__n_topics = n_topics
        
        #training
        if os.path.isfile(existing):
            # Load a potentially pretrained model from disk.
            self.model = LdaModel.load(temp_file)
        else:
            self.model = LdaModel(corpus=tqdm(self.__corpus),
                                      id2word=self.__id2word,
                                      num_topics=n_topics)
        # directory path
        self.__directory = directory
    
    @property
    def get_id2word(self):
        return self.__id2word
    
    @property
    def get_corpus(self):
        return self.__corpus
    
    def save_current_model(self):
        # retrieve time
        now = datetime.now()
        # create the directory if it doesn't exist
        try:
            os.makedirs(directory + now.strftime("%d%m%Y_%H%M%S"))
        except:
            pass
        # Save model to disk.
        temp_file = datapath(directory + now.strftime("%d%m%Y_%H%M%S") + '/model')
        
        self.model.save(temp_file)
        
    def get_perplexity(self):
        return self.model.log_perplexity(corpus)
    
    def get_coherence(self):
        coherence_model_lda = CoherenceModel(model=model, texts=documents, dictionary=id2word, coherence='c_v')
        return coherence_model_lda.get_coherence()
    
    
    # data vizualisation
    def dashboard_LDAvis(self):
        # some basic dataviz
        pyLDAvis.enable_notebook()
        vis = pyLDAvis.gensim.prepare(self.model, self.__corpus,
                                      dictionary = self.model.id2word)
        return vis
        
    def plot_tsne(self):
        # n-1 rows each is a vector with i-1 posisitons, where n the number of documents
        # i the topic number and tmp[i] = probability of topic i
        topic_weights = []
        for row_list in self.model[self.get_corpus]:
            tmp = np.zeros(self.__n_topics)
            for i, w in row_list:
                tmp[i] = w
            topic_weights.append(tmp)


        # Array of topic weights    
        arr = pd.DataFrame(topic_weights).fillna(0).values

        # Keep the well separated points (optional)
        arr = arr[np.amax(arr, axis=1) > 0.35]

        # Dominant topic number in each doc
        topic_num = np.argmax(arr, axis=1)
        print("topic_num :", topic_num)

        # tSNE Dimension Reduction
        tsne_model = TSNE(n_components=2, verbose=1, init='pca')
        tsne_lda = tsne_model.fit_transform(arr)

        # Plot the Topic Clusters using Bokeh
        output_notebook()
        mycolors = np.array([color for name, color in mcolors.TABLEAU_COLORS.items()])
        plot = figure(title="t-SNE Clustering of {} LDA Topics".format(self.__n_topics), 
                      plot_width=900, plot_height=700)
        plot.scatter(x=tsne_lda[:,0], y=tsne_lda[:,1], color=mycolors[topic_num])
        show(plot)
        
        
    

In [148]:
# create my model
lda_model = LDATopicModeling()

100%|██████████| 574/574 [00:00<00:00, 598.12it/s]


In [150]:
# print the result
lda_model.model.print_topics()

[(0,
  '0.021*"go" + 0.008*"to" + 0.008*"know" + 0.007*"day" + 0.006*"think" + 0.006*"love" + 0.006*"come" + 0.005*"tell" + 0.005*"want" + 0.005*"need"'),
 (1,
  '0.026*"love" + 0.019*"oh" + 0.018*"know" + 0.016*"go" + 0.016*"yeah" + 0.015*"to" + 0.013*"baby" + 0.012*"get" + 0.009*"girl" + 0.009*"yes"'),
 (2,
  '0.012*"help" + 0.011*"rock" + 0.010*"ba" + 0.008*"rattler" + 0.008*"love" + 0.007*"big" + 0.006*"wide" + 0.006*"want" + 0.006*"walk" + 0.006*"track"'),
 (3,
  '0.019*"get" + 0.014*"baby" + 0.013*"love" + 0.013*"know" + 0.012*"go" + 0.011*"come" + 0.010*"time" + 0.007*"oh" + 0.007*"man" + 0.006*"stand"'),
 (4,
  '0.018*"go" + 0.014*"to" + 0.014*"love" + 0.012*"know" + 0.012*"get" + 0.010*"like" + 0.009*"be" + 0.008*"come" + 0.007*"heart" + 0.006*"pity"'),
 (5,
  '0.052*"love" + 0.017*"know" + 0.013*"like" + 0.011*"go" + 0.009*"time" + 0.008*"to" + 0.008*"oh" + 0.007*"fall" + 0.007*"get" + 0.007*"come"'),
 (6,
  '0.063*"la" + 0.028*"ba" + 0.014*"da" + 0.011*"oop" + 0.009*"ann" + 

In [142]:
lda_model.plot_tsne()

topic_num : [8 2 5 5 2 4 5 2 6 2 1 7 3 8 4 0 7 9 2 1 5 6 2 0 1 1 3 3 4 0 2 2 7 1 3 1 4
 4 2 9 5 4 1 5 0 1 2 8 4 1 4 4 8 0 9 5 5 9 5 3 5 8 8 1 8 5 3 9 1 8 4 1 0 4
 1 9 2 0 7 7 3 4 2 6 5 5 5 5 4 7 1 0 5 3 3 4 4 6 4 8 3 9 3 8 8 9 1 9 7 4 4
 7 0 3 3 3 2 7 5 6 7 7 9 1 2 9 1 7 6 8 4 2 0 9 2 9 9 7 3 4 1 8 4 2 8 1 0 3
 9 3 4 1 2 5 4 6 3 1 0 3 1 1 7 1 3 1 3 5 7 1 1 5 1 5 0 0 8 4 8 4 8 4 1 1 8
 8 4 5 3 0 2 9 7 2 5 0 7 7 2 4 1 0 8 2 2 1 0 3 8 6 4 1 4 0 1 4 6 5 4 5 0 1
 3 0 1 6 4 3 0 2 1 0 4 4 1 0 1 3 8 0 0 9 5 0 0 9 4 4 4 0 4 7 1 3 8 0 3 2 1
 6 7 2 4 1 5 9 4 8 2 4 0 1 2 4 5 7 7 5 5 9 9 9 1 4 9 9 0 0 9 0 5 0 2 2 7 1
 1 6 7 0 5 1 2 3 3 7 1 2 7 1 6 2 1 4 8 3 2 9 0 1 2 9 4 0 2 6 3 0 6 5 0 7 4
 2 8 8 4 0 8 1 1 7 5 4 4 0 6 9 2 2 1 9 0 0 1 5 0 7 9 8 8 7 7 9 1 4 3 4 6 1
 1 4 9 9 0 2 0 3 2 1 7 7 0 4 2 6 4 9 1 7 0 5 8 7 6 0 0 3 0 1 4 1 3 1 4 0 4
 0 3 4 9 0 3 9 0 5 2 4 0 0 6 1 0 4 7 0 6 1 6 7 4 3 3 9 5 0 2 4 1 9 5 8 5 0
 7 7 7 1 5 9 5 7 1 8 3 8 0 9 3 6 2 0 0 9 7 0 7 5 0 7 3 7 0 7 5 2 0 9 3 9 2
 7 7 5 4 8 3 

/opt/conda/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:793: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.

/opt/conda/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:986: FutureWarning:

The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.



[t-SNE] KL divergence after 250 iterations with early exaggeration: 43.808891
[t-SNE] KL divergence after 1000 iterations: 0.137871


Loading BokehJS ...

In [155]:
def remove_folder_contents(folder):
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                remove_folder_contents(file_path)
                os.rmdir(file_path)
        except Exception as e:
            print(e)

folder_path = '/kaggle/working'
remove_folder_contents(folder_path)
os.rmdir(folder_path)


OSError: [Errno 16] Device or resource busy: '/kaggle/working'

In [156]:
lda_model.dashboard_LDAvis()

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.046576 -0.019843       1        1  15.137426
3      0.042408 -0.004066       2        1  14.696235
1      0.060674 -0.041134       3        1  13.099850
4      0.033566 -0.038251       4        1  11.116688
8     -0.012161 -0.017176       5        1   9.518438
9     -0.000406  0.014760       6        1   9.135528
6     -0.137823 -0.069691       7        1   8.109634
0     -0.008691  0.016820       8        1   7.479148
7      0.032397  0.047007       9        1   6.776358
2     -0.056539  0.111573      10        1   4.930695, topic_info=       Term        Freq       Total Category  logprob  loglift
1062     la  257.000000  257.000000  Default  30.0000  30.0000
583      ba  124.000000  124.000000  Default  29.0000  29.0000
18     love  808.000000  808.000000  Default  28.0000  28.0000
2715    oop   61.000000   61.000000  Default  27.0000  27.0000
2020     da   73.000000   73.000000  Default  26.0000  26.0000
...     ...         ...         ...      ...      ...      ...
213    know    9.472248  503.919701  Topic10  -5.3769  -0.9644
505     get    8.381674  341.390822  Topic10  -5.4992  -0.6973
1090  dream    6.238972  121.128330  Topic10  -5.7944   0.0437
105    come    6.056228  351.003917  Topic10  -5.8241  -1.0500
125    live    5.601685   95.689378  Topic10  -5.9022   0.1717

[761 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
517       5  0.160796    abide
517       6  0.803982    abide
2642      1  0.128697    adore
2642      2  0.042899    adore
2642      3  0.471889    adore
...     ...       ...      ...
3059      5  0.106513  youtake
3059      7  0.745588  youtake
9925      4  0.825603  youthru
9681      4  0.296185    zigga
9681      5  0.592370    zigga

[1878 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 4, 2, 5, 9, 10, 7, 1, 8, 3])

## 2015 songs lyrics topic modelling

Let's first retrieve the english song in 2015 (year of with max genius lyrics repertoried).

Let's plot the tag distribution

The barplot below shows the frequency of each tag color by total views

Let's try topic modelling with top2vec library which is the easiest to start. But first let's filter the lyrics.

# 2000 song lyrics topic modelling

## References